# Finetuning SDXL Dreambooth using AutoTrain
Introduction to Generative AI Workshop

© SkillSurf 2024

In [1]:
#@title How to run this notebook
#@markdown This notebook was created using the reference published by HuggingFace.
#@markdown - **Step 1** - Upload images to a folder named `images/`
#@markdown - **Step 2** - Choose a project name (optional)
#@markdown - **Step 3** - Update the prompt with a unique identifier (Choose a word that doesn't appear in dictionaries)
#@markdown - **Step 4** - Run all cells in order \\

#@markdown For original notebook/code visit:https://github.com/huggingface/autotrain-advanced

## Installing AutoTrain

In [ ]:
# Install AutoTrain using pip
import os
!pip install -U autotrain-advanced

## Setting up a training session

In [3]:
#@markdown ---
#@markdown #### Project Config
project_name = 'skillsurfGenAI' # @param {type:"string"}
model_name = 'stabilityai/stable-diffusion-xl-base-1.0' # @param ["stabilityai/stable-diffusion-xl-base-1.0", "runwayml/stable-diffusion-v1-5", "stabilityai/stable-diffusion-2-1", "stabilityai/stable-diffusion-2-1-base"]
prompt = 'cartoon of a sks shark' # @param {type: "string"}

#@markdown ---
#@markdown #### Hyperparameters
learning_rate = 1e-4 # @param {type:"number"}
num_steps = 500 #@param {type:"number"}
batch_size = 1 # @param {type:"slider", min:1, max:32, step:1}
gradient_accumulation = 4 # @param {type:"slider", min:1, max:32, step:1}
resolution = 1024 # @param {type:"slider", min:128, max:1024, step:128}
use_8bit_adam = False # @param ["False", "True"] {type:"raw"}
use_xformers = False # @param ["False", "True"] {type:"raw"}
mixed_precision = "fp16" # @param ["fp16", "bf16", "none"] {type:"raw"}
train_text_encoder = False # @param ["False", "True"] {type:"raw"}
disable_gradient_checkpointing = False # @param ["False", "True"] {type:"raw"}

os.environ["PROJECT_NAME"] = project_name
os.environ["MODEL_NAME"] = model_name
os.environ["PROMPT"] = prompt
os.environ["LEARNING_RATE"] = str(learning_rate)
os.environ["NUM_STEPS"] = str(num_steps)
os.environ["BATCH_SIZE"] = str(batch_size)
os.environ["GRADIENT_ACCUMULATION"] = str(gradient_accumulation)
os.environ["RESOLUTION"] = str(resolution)
os.environ["USE_8BIT_ADAM"] = str(use_8bit_adam)
os.environ["USE_XFORMERS"] = str(use_xformers)
os.environ["MIXED_PRECISION"] = str(mixed_precision)
os.environ["TRAIN_TEXT_ENCODER"] = str(train_text_encoder)
os.environ["DISABLE_GRADIENT_CHECKPOINTING"] = str(disable_gradient_checkpointing)

In [7]:
# Run this cell to avoid errors
!rmdir /content/images/.ipynb_checkpoints

In [8]:
# Run the AutoTrain CLI command using parameteres given above
!autotrain dreambooth \
--model ${MODEL_NAME} \
--project-name ${PROJECT_NAME} \
--image-path images/ \
--prompt "${PROMPT}" \
--resolution ${RESOLUTION} \
--batch-size ${BATCH_SIZE} \
--num-steps ${NUM_STEPS} \
--gradient-accumulation ${GRADIENT_ACCUMULATION} \
--lr ${LEARNING_RATE} \
--mixed-precision ${MIXED_PRECISION} \
$( [[ "$USE_XFORMERS" == "True" ]] && echo "--xformers" ) \
$( [[ "$TRAIN_TEXT_ENCODER" == "True" ]] && echo "--train-text-encoder" ) \
$( [[ "$USE_8BIT_ADAM" == "True" ]] && echo "--use-8bit-adam" ) \
$( [[ "$DISABLE_GRADIENT_CHECKPOINTING" == "True" ]] && echo "--disable_gradient-checkpointing" ) \

INFO     | 2024-06-03 05:12:42 | autotrain.cli.run_dreambooth:run:388 - Running DreamBooth Training
WARNING  | 2024-06-03 05:12:42 | autotrain.trainers.common:__init__:180 - Parameters supplied but not used: func, version, train, log, inference, train_split, config, data_path, backend, deploy, valid_split
INFO     | 2024-06-03 05:12:42 | autotrain.preprocessor.dreambooth:_save_concept_images:67 - Saving concept images
INFO     | 2024-06-03 05:12:42 | autotrain.preprocessor.dreambooth:_save_concept_images:68 - images/1.jpg
INFO     | 2024-06-03 05:12:42 | autotrain.preprocessor.dreambooth:_save_concept_images:67 - Saving concept images
INFO     | 2024-06-03 05:12:42 | autotrain.preprocessor.dreambooth:_save_concept_images:68 - images/2.jpg
INFO     | 2024-06-03 05:12:42 | autotrain.preprocessor.dreambooth:_save_concept_images:67 - Saving concept images
INFO     | 2024-06-03 05:12:42 | autotrain.preprocessor.dreambooth:_save_concept_images:68 - images/5.jpg
INFO     | 2024-06-03 05:12:42

## Save model weights to Google Drive

In [ ]:
# Mount Google Drive onto Colab Folder
from google.colab import drive
drive.mount('/content/drive')

In [3]:
# Copy session folder into a folder in the drive
!cp -r /content/skillsurfGenAI /content/drive/MyDrive/Dreambooth

# Generating images

Now that we have trained a large model, our resources have been occupied. We should free the occupied RAM and GPU RAM before generating images. To do that, restart the runtime session and proceed with the cells below.

### Importing modules and building the Stable Diffusion pipeline

In [ ]:
from diffusers import DiffusionPipeline
import torch

prj_path = "/content/skillsurfGenAI"
model = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = DiffusionPipeline.from_pretrained(
    model,
    torch_dtype=torch.float16,
)
pipe.to("cuda")
pipe.load_lora_weights(prj_path, weight_name="pytorch_lora_weights.safetensors")

### Generating images for your own prompt

In [ ]:
prompt = """cartoon of a sks shark as a Jedi holding a lightsaber, Star Wars, hyper detailed background, character concept, full body, dynamic pose, intricate,
            highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, square thumbnail, 4k, uhd,
            art by artgerm and greg rutkowski and alphonse mucha"""

seed = 42
generator = torch.Generator("cuda").manual_seed(seed)
image = pipe(prompt=prompt, generator=generator).images[0]
image.save(f"4.png")

## (Optional) - Refining the generated images


In [ ]:
# Import refiner from Stable Diffusion
from diffusers import StableDiffusionXLImg2ImgPipeline

# Initialize refiner
refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-refiner-1.0",
    torch_dtype=torch.float16,
)
refiner.to("cuda")

# Create new prompt
prompt = "cartoon of a sks shark riding a formula one race car, comic art, anime style, 4k, uhd"

# Generate image
seed = 42
generator = torch.Generator("cuda").manual_seed(seed)
image = pipe(prompt=prompt, generator=generator).images[0]

# Refine the image
image = refiner(prompt=prompt, generator=generator, image=image).images[0]

image.save(f"3.png")